In [25]:
import pandas as pd
import os

In [26]:
df_calls = pd.read_csv('/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/clean/megaline_calls_clean.csv')
df_messages = pd.read_csv('/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/clean/megaline_messages_clean.csv')
df_internet = pd.read_csv('/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/clean/megaline_internet_clean.csv')
df_users = pd.read_csv('/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/clean/megaline_users_clean.csv')
df_plans = pd.read_csv('/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/clean/megaline_plans_clean.csv')
df_churn = pd.read_csv('/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/clean/churn_clean.csv')

In [31]:
# Calcular total_messages, total_gb y total_minutes por usuario y mes
messages_count = df_messages.groupby(["user_id", "month"]).size().reset_index(name="total_messages")
data_usage_sum = df_internet.groupby(["user_id", "month"])["mb_used"].sum().reset_index(name="total_gb")
calls_sum = df_calls.groupby(["user_id", "month"])["duration"].sum().reset_index(name="total_minutes")

# Unir todas las métricas en un solo DataFrame
features_df = messages_count.merge(data_usage_sum, on=["user_id", "month"], how="outer")
features_df = features_df.merge(calls_sum, on=["user_id", "month"], how="outer")
features_df = features_df.merge(df_users, on="user_id", how="left")

# Unir con la información de los planes
features_df = features_df.merge(df_plans, left_on="plan", right_on="plan_name", how="left")

# Reemplazar valores NaN con 0 en las métricas calculadas
features_df.fillna(0, inplace=True)

# Calcular el costo adicional solo si se excede el límite
features_df["extra_messages"] = (features_df["total_messages"] - features_df["messages_included"]).clip(lower=0)
features_df["extra_gb"] = (features_df["total_gb"] - (features_df["mb_per_month_included"] / 1024)).clip(lower=0)
features_df["extra_minutes"] = (features_df["total_minutes"] - features_df["minutes_included"]).clip(lower=0)

# Calcular el costo total teniendo en cuenta el plan base y los cargos extra
features_df["total_cost"] = (
    features_df["usd_monthly_pay"] +
    (features_df["extra_messages"] * features_df["usd_per_message"]) +
    (features_df["extra_gb"] * features_df["usd_per_gb"]) +
    (features_df["extra_minutes"] * features_df["usd_per_minute"])
) / 1000

In [30]:
print(features_df[["usd_per_message", "usd_per_gb", "usd_per_minute"]].describe())


       usd_per_message   usd_per_gb  usd_per_minute
count      2293.000000  2293.000000     2293.000000
mean          0.023720     9.058003        0.023720
std           0.009284     1.392652        0.009284
min           0.010000     7.000000        0.010000
25%           0.010000     7.000000        0.010000
50%           0.030000    10.000000        0.030000
75%           0.030000    10.000000        0.030000
max           0.030000    10.000000        0.030000


In [32]:
folder_path = "/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/ml"

file_path = os.path.join(folder_path, "facts_total_cost_by_month.csv")

features_df.to_csv(file_path, index=False)

print("Feature engineering completado y guardado en facts_total_cost_by_month.csv")


Feature engineering completado y guardado en facts_total_cost_by_month.csv


In [29]:
folder_path = "/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/ml"
file_path = os.path.join(folder_path, "dim_calls.csv")
df_calls.to_csv(file_path, index=False)

folder_path = "/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/ml"
file_path = os.path.join(folder_path, "dim_messages.csv")
df_messages.to_csv(file_path, index=False)

folder_path = "/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/Alvarez. Parcial1/data/ml"
file_path = os.path.join(folder_path, "dim_internet.csv")
df_internet.to_csv(file_path, index=False)